Johana Tellez-Michael Caicedo
##Descripción general del algoritmo

El algoritmo K-Nearest Neighbors (K-NN) es un método de clasificación basado en vecindad. Para clasificar un nuevo punto, busca los k ejemplos más cercanos en el conjunto de entrenamiento y asigna la clase más común entre ellos.
Es un modelo no paramétrico (no aprende una función explícita), simple de implementar y efectivo cuando la estructura espacial de los datos refleja las clases.

En este contexto, utilizaremos K-NN para aproximar la política aprendida por el agente Q-Learning. La idea es extraer pares (estado, mejor acción) desde la tabla Q entrenada, y entrenar un K-NN que imite esas decisiones. Luego, el modelo puede sustituir o complementar a la política original en la toma de decisiones.

##1. CLASES DEL ENTORNO Y AGENTE

En esta sección se definen las estructuras fundamentales del experimento de aprendizaje por refuerzo:

PongEnvironment: es la clase que simula el juego de Pong. Define el espacio de estados (posición del jugador y de la pelota), las acciones posibles (mover arriba o abajo), las reglas del juego (rebotes, colisiones, pérdida de vidas) y entrega las recompensas en cada paso.

PongAgent: representa al agente que aprende a jugar. Contiene la tabla Q (donde guarda su política), el método para decidir acciones (get_next_step) y la función para actualizar su conocimiento mediante la ecuación de Bellman (update).

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from random import randint
from math import ceil, floor
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# ==========================
# 1. CLASES DEL ENTORNO Y AGENTE
# ==========================
class PongEnvironment:
    def __init__(self, max_life=3, height_px=40, width_px=50, movimiento_px=3):
        self.action_space = ['Arriba', 'Abajo']
        self._step_penalization = 0
        self.state = [0,0,0]
        self.total_reward = 0
        self.dx = movimiento_px
        self.dy = movimiento_px
        filas = ceil(height_px/movimiento_px)
        columnas = ceil(width_px/movimiento_px)
        self.positions_space = np.array([[[0 for z in range(columnas)] for y in range(filas)] for x in range(filas)])
        self.lives = max_life
        self.max_life = max_life
        self.x = randint(int(width_px/2), width_px)
        self.y = randint(0, height_px-10)
        self.player_alto = int(height_px/4)
        self.player1 = self.player_alto
        self.score = 0
        self.width_px = width_px
        self.height_px = height_px
        self.radio = 2.5

    def reset(self):
        self.total_reward = 0
        self.state = [0,0,0]
        self.lives = self.max_life
        self.score = 0
        self.x = randint(int(self.width_px/2), self.width_px)
        self.y = randint(0, self.height_px-10)
        return self.state

    def step(self, action, animate=False):
        self._apply_action(action, animate)
        done = self.lives <= 0
        reward = self.score + self._step_penalization
        self.total_reward += reward
        return self.state, reward, done

    def _apply_action(self, action, animate=False):
        if action == "Arriba":
            self.player1 += abs(self.dy)
        elif action == "Abajo":
            self.player1 -= abs(self.dy)
        self.avanza_player()
        self.avanza_frame()
        self.state = (floor(self.player1/abs(self.dy))-2, floor(self.y/abs(self.dy))-2, floor(self.x/abs(self.dx))-2)

    def detectaColision(self, ball_y, player_y):
        return (player_y+self.player_alto >= (ball_y-self.radio)) and (player_y <= (ball_y+self.radio))

    def avanza_player(self):
        if self.player1 + self.player_alto >= self.height_px:
            self.player1 = self.height_px - self.player_alto
        elif self.player1 <= -abs(self.dy):
            self.player1 = -abs(self.dy)

    def avanza_frame(self):
        self.x += self.dx
        self.y += self.dy
        if self.x <= 3 or self.x > self.width_px:
            self.dx = -self.dx
            if self.x <= 3:
                ret = self.detectaColision(self.y, self.player1)
                if ret:
                    self.score = 10
                else:
                    self.score = -10
                    self.lives -= 1
                    if self.lives > 0:
                        self.x = randint(int(self.width_px/2), self.width_px)
                        self.y = randint(0, self.height_px-10)
                        self.dx = abs(self.dx)
                        self.dy = abs(self.dy)
        else:
            self.score = 0
        if self.y < 0 or self.y > self.height_px:
            self.dy = -self.dy

class PongAgent:
    def __init__(self, game, policy=None, discount_factor=0.1, learning_rate=0.1, ratio_explotacion=0.9):
        if policy is not None:
            self._q_table = policy
        else:
            position = list(game.positions_space.shape)
            position.append(len(game.action_space))
            self._q_table = np.zeros(position)
        self.discount_factor = discount_factor
        self.learning_rate = learning_rate
        self.ratio_explotacion = ratio_explotacion

    def get_next_step(self, state, game):
        next_step = np.random.choice(list(game.action_space))
        if np.random.uniform() <= self.ratio_explotacion:
            idx_action = np.random.choice(np.flatnonzero(
                self._q_table[state[0], state[1], state[2]] ==
                self._q_table[state[0], state[1], state[2]].max()
            ))
            next_step = list(game.action_space)[idx_action]
        return next_step

    def update(self, game, old_state, action_taken, reward_action_taken, new_state, reached_end):
        idx_action_taken = list(game.action_space).index(action_taken)
        actual_q_value = self._q_table[old_state[0], old_state[1], old_state[2], idx_action_taken]
        future_q_value_options = self._q_table[new_state[0], new_state[1], new_state[2]]
        future_max_q_value = reward_action_taken + self.discount_factor * future_q_value_options.max()
        if reached_end:
            future_max_q_value = reward_action_taken
        self._q_table[old_state[0], old_state[1], old_state[2], idx_action_taken] = actual_q_value + \
            self.learning_rate * (future_max_q_value - actual_q_value)

    def get_policy(self):
        return self._q_table





##FUNCIÓN DE ENTRENAMIENTO
En esta sección se define la función play(), que se encarga de entrenar al agente Pong usando el algoritmo Q-Learning.

Lo que hace esta función es:

Reiniciar el entorno y la posición de la pelota en cada partida.

Permitir que el agente interactúe con el entorno durante muchas rondas (por ejemplo, 3000 o 5000 partidas).

En cada paso, el agente elige una acción, recibe una recompensa y actualiza su tabla Q para mejorar su política.

Controla la duración de las partidas y registra la recompensa total.

In [7]:
# ==========================
# 2. FUNCIÓN DE ENTRENAMIENTO
# ==========================
def play(rounds=5000, max_life=3, discount_factor=0.1, learning_rate=0.1,
         ratio_explotacion=0.9, learner=None, game=None, animate=False):
    if game is None:
        game = PongEnvironment(max_life=max_life, movimiento_px=3)
    if learner is None:
        learner = PongAgent(game, discount_factor=discount_factor, learning_rate=learning_rate, ratio_explotacion=ratio_explotacion)

    for _ in range(rounds):
        state = game.reset()
        done = False
        while not done and game.total_reward <= 1000:
            old_state = np.array(state)
            next_action = learner.get_next_step(state, game)
            state, reward, done = game.step(next_action, animate=animate)
            learner.update(game, old_state, next_action, reward, state, done)
    return learner, game



## 3. ENTRENAR AGENTE Y MODELO KNN
En esta sección se realiza todo el flujo de entrenamiento tanto del agente como del modelo supervisado K-NN:

Entrenar al agente con Q-Learning utilizando la función play(). Esto permite generar una tabla Q con la política aprendida a partir de la experiencia.

Extraer la Q-table y convertirla en un dataset supervisado, donde:

Cada fila representa un estado del juego (posición del jugador, posición Y y X de la pelota).

La etiqueta es la mejor acción para ese estado (según la política aprendida).

Dividir el dataset en entrenamiento y validación.

Entrenar un modelo K-Nearest Neighbors (KNN) con los datos generados, para que aprenda a imitar la política.

Evaluar el modelo con métricas de precisión, recall y f1-score, verificando qué tan bien reproduce las decisiones del agente.

In [8]:
# ==========================
# 3. ENTRENAR AGENTE Y MODELO KNN
# ==========================
learner, game = play(rounds=3000, discount_factor=0.1, learning_rate=0.1, ratio_explotacion=0.9)

q_table = learner.get_policy()
pos_n, y_n, x_n, n_actions = q_table.shape

X = []
y = []
for i in range(pos_n):
    for j in range(y_n):
        for k in range(x_n):
            best_action_idx = int(np.argmax(q_table[i, j, k]))
            X.append([i, j, k])
            y.append(best_action_idx)

X = np.array(X)
y = np.array(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_val)
print("Exactitud KNN:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))

Exactitud KNN: 0.6536731634182908
              precision    recall  f1-score   support

           0       0.69      0.71      0.70       381
           1       0.60      0.58      0.59       286

    accuracy                           0.65       667
   macro avg       0.65      0.64      0.65       667
weighted avg       0.65      0.65      0.65       667



##Análisis de resultados — K-NN

El modelo K-Nearest Neighbors alcanzó una exactitud del 65,8 %, con una precisión y recall equilibrados entre ambas clases de acción (Arriba y Abajo). La clase 0 (Arriba) presentó un desempeño ligeramente superior (precisión 0.70), lo que indica que el clasificador identifica mejor las situaciones en las que la acción correcta es moverse hacia arriba.

Estos resultados son razonables, considerando que el espacio de estados del entorno Pong es discreto y que K-NN utiliza una métrica de vecindad simple sin ponderaciones adicionales. El modelo logra capturar de forma aproximada la política aprendida por Q-Learning, aunque con un margen de error visible que podría deberse a estados menos frecuentes en el entrenamiento o a acciones ambiguas cerca de los bordes.

En resumen, el K-NN constituye un modelo sustituto funcional, capaz de reproducir la política con una precisión aceptable, ofreciendo simplicidad y rapidez sin requerir un entrenamiento complejo.